# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a classification problem as we are trying to put a label on the students. Are they at risk of failing or not. If so then precautions and measures will be taken by the school to help those in need.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%age)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1  # not counting the last column as it is the label
n_passed = (student_data["passed"]=="yes").sum()
n_failed = (student_data["passed"]=="no").sum()
grad_rate = 100*float(n_passed)/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


In [6]:
X_all.shape

(395, 30)

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all, 
    test_size=num_test, 
    train_size=num_train, 
    random_state=42)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [9]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.008


In [10]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.876068376068


In [11]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.783783783784


### SVC

**Complexity**

Complexity varies between $O(n_{features}\ x\ n_{samples}^2)$ and $O(n_{features}\ x\ n_{samples}^3)$.
In our case, the number of sample is 395 and the number of features (after transformation) is 48. Because it is an eager learner the training time will grow as the dataset grows. It will grow exponentially fast.

If the dataset was sparse we could reduce the complexity, replacing $n_{features}$ by the average number of non zero values for all data points.

Prediction time is constant as we are just letting the data point go through the learned formula, $O(1)$.

The space required is $O(n_{samples})$ for the training and constant for predicting as we only keep a few parameters to make the prediction, so $O(1)$


**The advantages of support vector machines are**:
- Effective in high dimensional spaces.
- Still effective in cases where number of dimensions is greater than the number of samples.
- Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
- Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

**The disadvantages of support vector machines include**:
- If the number of features is much greater than the number of samples, the method is likely to give poor performances.
- SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation.

http://scikit-learn.org/stable/modules/svm.html#complexity


**Why this model**:
- We usually have to worry about the curse of dimensionnality where we need more and more data with high dimensions. Given the train size we have to try, as low as 100, it brings the number of datapoints closer to the number of features which would contradict the recommendations to avoid the curse of dimensionnality. But because SVM deals well in those circumstances I chose to try it. 

http://scikit-learn.org/stable/modules/svm.html#

In [12]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Note: Keep the test set constant
def train_on_train_size(clf, train_size=xrange(100,400,100)):
    for i in train_size:
        X_train, X_test, y_train, y_test = train_test_split(
            X_all, 
            y_all, 
            test_size=num_test, 
            train_size=i, 
            random_state=42)

        train_predict(clf, X_train, y_train, X_test, y_test)
clf = SVC()
train_on_train_size(clf)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.877697841727
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.774647887324
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.867924528302
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.781456953642
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.876068376068
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.783783783784


### kNN

**Complexity**

The complexity of a prediction in kNN, as we saw in class, is $k + log(n)$ if the input feature is one dimensionnal and ordered. Searching where a number fits is $log(n)$ and we have to do $k-1$ more comparison. So $k + log(n)$ because $k$ can be a function of $n$.

But we don't have a one dimensional input feature, so ordering is sort of meaningless. Even though there are schemes to reduce the search of inserting one point in a multidimensional space, I will consider the worst case scenario. We need to have a measure of distance or similarity between two points which will be done necessarily by using all the dimensions/features of both points, wether it's euclidian distance or manhattan or any other, so $O(n_{features})$. Then we need to compare every points to the one we are inserting. Fortunately we can keep the calculated distance as we go along and keep the $k$ top closest. $O(n_{samples})$ comparison are required. For a total of $O(n_{features}*n_{samples})$

The complexity of training, as we also saw in class is constant, $O(1)$

The space required for training and predicting is the size of the dataset, $O(n_{samples})$

**The advantages of kNN**
- Really fast training time. In fact, no time at all since it's a lazy learner. It just sits there waiting to be queried for predictions. No matter what the size of the training set is, it's a constant learning rate.
- Easy to add data points for training. If we want to add new samples we don't have to retrain anything.
- It works well on noisy data that is not easily separable.
- Not affected by $k$ (depending on the choice of algorithm, but we are talking about brute force here) because we are already looking at all the datapoints.
- Simple and easy to understand

**The disadvantages of kNN**
- Prediction is dependent on the sample size. The bigger it is, the more comparison we have to do.
- It is not clear what is the best value for $k$
- It is not clear what is the best distance metric
- Being distance based, all the features contribute equally which could be a bad thing if some are useless

**Why this model**
- Despite its simplicity, kNN is well respected
- I wasn't sure how well the classes were seperable and kNN works well in those cases
- A lot of features are binary so they mostly sit on the unit sphere and most of the features are will contribute more or less equally with some exception such as age and absence.


In [13]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

train_on_train_size(clf)

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.003
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.805970149254
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.724637681159
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.000
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.88
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.769230769231
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.88

### AdaBoost

**Complexity**

Using boosting with decision stump (decision tree with depth 1). A decision tree of depth 1 will have to analyze $n_{features}$ to find the one with the highest information gain (IG). It needs to go through $n_{samples}$ to calculate the IG and do that $n_{estimators}$ times. Total complexity is $O(n_{features}n_{samples}n_{estimators})$

Prediction time depends on the underlying weak learner. In this case we have a decision tree. For every weak classifier we have to look at the selected feature for every node down the tree. So that would be one node per depth level and in this case just one node, the root. We have $N$ weak learners going down depth one, so it's a constant time for prediction, $O(1)$.

**The advantages of AdaBoost**
- As we add more and more weak learners, the train and test error go down. Test error would typically start going up after a given complexity (here complexity is the number of weak learners) but not in the case of boosting.
- Training weak learners is much faster than training strong learners.
- If the weak learners do slightly better than random, they become a powerhouse when combined.

**The disadvantages of AdaBoost**
- Becomes a bit of a black box. Hard to interpret the results
- *"if any non-zero fraction of the training data is mis-labeled, the boosting algorithm tries extremely hard to correctly classify these training examples, and fails to produce a model with accuracy better than 1/2."*
http://www.cs.columbia.edu/~rocco/Public/mlj9.pdf

**Why this model**
- The training speed is relatively fast compared to strong learners such as neural networks. Since this is a requirement from the board it makes sense to explore the fast options first.
- Boosting is also quite simple to explain to a board.
- From previous experience boosting often gave among the best results
- Using Decision Tree as weak learner because most of the features are binary and calculating the information is a bit faster

In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(criterion='entropy', max_depth=1))

train_on_train_size(clf)

------------------------------------------
Training set size: 100
Training AdaBoostClassifier...
Done!
Training time (secs): 0.074
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.932330827068
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.764705882353
------------------------------------------
Training set size: 200
Training AdaBoostClassifier...
Done!
Training time (secs): 0.101
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.885813148789
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.793893129771
------------------------------------------
Training set size: 300
Training AdaBoostClassifier...
Done!
Training time (secs): 0.076
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.88127853

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

I would chose SVC with a training set size of 100. Among the three train set sizes that we tried, F1 score stays approximatly the same and is on par with Adaboost. Both these algorithm beat kNN on that score. Training time is not as fast as kNN, which basically has no training time, but an order of magnitude faster that Adaboost. Prediction time is fast as it is not distance based like kNN.

So at 100 train set size, SVC has the best performance and is among the fastest to train. A small prediction time is also important as we will be running all the students through it. It is similar to a database where we say "write once read many times". Once train it also takes almost no memory has it only needs to use a few parameters.

- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).


Our objective is to seperate the students that are at risk of failing on one side and those who are safe on the other side. This is no simple task as many parameters are intervening in this classification. We are trying to find a line that will best make this separation between those at risk and those who are not with a margin as large as possible. Meaning that we want to find a line that sits as comfortably as possible between the two groups without being closer to one or the other.

Intuitiely we thing about a straight line, but such a line might not allow us to make the best separation as the two classes might be mixed up a little bit. So we bring the problem to a higher dimension (from m dimension to n dimension where n > m). In this new space, a straight line (hyperplane) might exists where it is much simpler to make a clean separation. So classifying a new student we will bring him to this higher dimension and see on what side of the line it falls.

- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.


In [15]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
parameters = {'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.1, 1, 10, 100], 'gamma':[0.001, 0.1, 10, 100]}
clf = GridSearchCV(
    SVC(), 
    parameters, 
    scoring=make_scorer(f1_score, pos_label='yes'))
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf', 'sigmoid'), 'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)

- What is the model's final F<sub>1</sub> score?

In [18]:
y_pred = clf.best_estimator_.predict(X_test)
print "F1 score: {}".format(f1_score(y_test, y_pred, pos_label='yes'))
print "Best params: {}".format(clf.best_params_)

F1 score: 0.783783783784
Best params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
